In [1]:
import sys
sys.path.append("..")

from libs.InteractivePitchUI import *
from libs.data_manipulation import *
from libs.dim_reduction import *
from libs.feature_generation import *
from libs.clustering import *
from libs.convex_hull import *
from libs.alpha_shape import *
from libs.weight_generator import *
from libs.similar_movement import *
from libs.Video_Player import *

/home/martin/.local/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (None)/charset_normalizer (2.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
matches = compile_team_tracking_data_with_labels("../data/subset", "Denmark", "../data/Labelled_ground_truths.csv")
df = extract_one_match(matches, 1)


In [3]:
df.head()

,match_id,Time [s],Time [s]_team,half_team,minute_team,Period_team,home_3_x,home_3_y,home_10_x,home_10_y,...,away_16_x,away_16_y,ball_x,ball_y,ball_z,match_name,Label,Time[s],Competition,half_y
0,1,0.00,0.00,1H,0,1,18.652441,5.887125,6.911937,-6.848659,...,NaN,NaN,0.024191,-0.002120,0.135756,Denmark_England,Missing,NaN,NaN,NaN
1,1,0.04,0.04,1H,0,1,18.658504,5.907613,6.895243,-6.852221,...,NaN,NaN,-0.905583,-0.024639,0.151328,Denmark_England,Missing,NaN,NaN,NaN
2,1,0.08,0.08,1H,0,1,18.665127,5.926075,6.877546,-6.857188,...,NaN,NaN,-1.901920,-0.042994,0.161823,Denmark_England,Missing,NaN,NaN,NaN
3,1,0.12,0.12,1H,0,1,18.671473,5.942472,6.857270,-6.863159,...,NaN,NaN,-2.872990,-0.058094,0.147205,Denmark_England,Missing,NaN,NaN,NaN
4,1,0.16,0.16,1H,0,1,18.678909,5.958709,6.832435,-6.869350,...,NaN,NaN,-3.809885,-0.062621,0.143380,Denmark_England,Missing,NaN,NaN,NaN


In [4]:
func_control = lambda x:1
func_inverse = lambda x: 20/x
func_linear = lambda x: 200-x
func_exp = lambda x: np.exp(-x/20)

In [ ]:
import datetime


def process_situations(df):
    # Dictionary to store recommendations
    recommendations_dict = {}

    # Step 2: Extract relevant rows (all rows with a label not equal to "Missing")
    labeled_rows = df[df['Label'] != "Missing"]
    relevant_indices = labeled_rows.index.tolist()

    # Step 2: Process each relevant index
    # Step 2: Process each relevant index
    for relevant_index in relevant_indices:
        # Step 3: Apply Wasserstein distance for various functions upfront
        finished_indices_control = most_similar_with_wasserstein(relevant_index, df, func_control)
        finished_indices_linear = most_similar_with_wasserstein(relevant_index, df, func_linear)
        finished_indices_inverse = most_similar_with_wasserstein(relevant_index, df, func_inverse)
        finished_indices_exp = most_similar_with_wasserstein(relevant_index, df, func_exp)

        # Convert to numpy arrays
        finished_indices_control = np.array(finished_indices_control)
        finished_indices_exp = np.array(finished_indices_exp)
        finished_indices_linear = np.array(finished_indices_linear)
        finished_indices_inverse = np.array(finished_indices_inverse)

        # Step 4: Filter indices based on the time window (±480 seconds)
        time_window_filter = lambda indices: indices[
            (indices > relevant_index + 480) | 
            (indices < relevant_index - 480)
        ]

        finished_indices_control = time_window_filter(finished_indices_control)[:100]
        finished_indices_exp = time_window_filter(finished_indices_exp)[:100]
        finished_indices_linear = time_window_filter(finished_indices_linear)[:100]
        finished_indices_inverse = time_window_filter(finished_indices_inverse)[:100]

        # Combine recommendations into a dictionary for separate processing
        wasserstein_recommendations = {
            "Control": finished_indices_control,
            "Exp": finished_indices_exp,
            "Linear": finished_indices_linear,
            "Inverse": finished_indices_inverse
        }
        # Create the folder based on the label
        # Step 6: Prepare the result DataFrame for this function set
        label = df.loc[relevant_index, 'Label']  # Get the label for the situation
        match_name = df.loc[relevant_index, 'match_name']
        time_of_situation = str(datetime.timedelta(seconds=df.loc[relevant_index, 'Time [s]']))

        folder_name = f"{label}_{match_name}_{time_of_situation}"
        generate_folder(folder_name)

        # Save the time and match_name to a text file
        with open(f"{folder_name}/situation_info.txt", "w") as f:
                f.write(f"Match Name: {match_name}\n")
                f.write(f"Time of Situation: {time_of_situation}\n")

        # Step 5: Process each set of indices separately
        for func_name, indices_set in wasserstein_recommendations.items():
            print(indices_set, func_name)
            if len(indices_set) == 0:
                continue  # Skip if no indices in this set

            # Step 5.1: Compute similar movements for the current set
            similar_movement = find_similar_movement_entire_team(df, relevant_index, indices_set,ball_weight=0.9)
            similar_movement_ranked = sorted(similar_movement, key=lambda x: x[0])

            # Step 5.2: Select the top 50 indices and add relevant indices
            new_indices = [index for _, index in similar_movement_ranked][:10]
            new_df = df.loc[new_indices]

            
            


            # Generate the result DataFrame
            result_df = new_df[["match_name", "Time [s]", "half_team"]]
            result_df["Time [s]"] = result_df["Time [s]"].apply(lambda x: str(datetime.timedelta(seconds=x)))

            # Save the ground truth CSV for this function set
            result_df.to_csv(f"{folder_name}/{folder_name}_{func_name}_ground_truth.csv")

            # Shuffle and save the ordering truth CSV for this function set
            result_df.sample(frac=1).to_csv(f"{folder_name}/{folder_name}_{func_name}_for_ordering_truth.csv")

            # Store the recommendations in the dictionary
            recommendations_dict[relevant_index] = recommendations_dict.get(relevant_index, {})
            recommendations_dict[relevant_index][func_name] = new_indices

    # Return the dictionary with recommendations
    return recommendations_dict

    

reccomendations_dict = process_situations(df)

In [5]:
df = pd.read_csv("breakthrough-space_Denmark_England_0:25:40/breakthrough-space_Denmark_England_0:25:40_Exp_for_ordering_truth.csv")

# Function to convert time string to seconds
def convert_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

# Apply the conversion to the column
df["Time [s]"] = df["Time [s]"].apply(convert_to_seconds)





df.columns
list  = df[["Time [s]", "Unnamed 0"]].to_numpy()


print(indices_1)
VideoPlayer("3-2024-2036178-Denmark_England.mp4",8,indices_1)

ValueError: invalid literal for int() with base 10: '06.760000'

In [ ]:
import vlc
import time
import tkinter as tk
from tkinter import Button, Label
import numpy as np

# Path to your video file
video_path = "/data/video/EURO2020Russia_vs_Denmark_1st_Half_769630.mp4"

# Video offset in seconds (adjust this value to align video and match time)
video_offset = 52  # Example: If the video starts 2 minutes before the actual match starts

# Create a VLC instance
vlc_instance = vlc.Instance()

# Create a VLC media player
player = vlc_instance.media_player_new()

# Load the media file
media = vlc_instance.media_new(video_path)
player.set_media(media)

# Pause the video initially
player.play()
time.sleep(2)  # Wait for the video to load
player.pause()

# Function to seek to a specific time in seconds
def seek_to_time(player, seconds):
    if isinstance(seconds, (int, float)):  # Ensure that the time is a scalar number
        player.set_time(int((seconds + video_offset) * 1000))  # Add video_offset to align with match time

# Pause the video
def pause_video():
    player.pause()

# Resume the video
def play_video():
    player.play()

# Close the video properly and destroy the tkinter window
def close_video():
    if player.is_playing():
        player.stop()
    player.release()
    root.destroy()  # Close the tkinter window gracefully

# Function to update the current time in the video player
def update_time_label():
    current_time = player.get_time() // 1000  # Get current time in milliseconds, convert to seconds
    time_label.config(text=f"Time: {current_time} seconds")
    root.after(1000, update_time_label)  # Update every second

# Function to skip to the next closest situation time
def next_time():
    global current_index
    if len(distance_index_list) > 0:
        current_index = (current_index + 1) % len(distance_index_list)
        specific_time_in_seconds = distance_index_list[current_index][0]  # Extract the correct time from closest situations
        seek_to_time(player, specific_time_in_seconds)
        time_label.config(text=f"Time: {specific_time_in_seconds} seconds")

# Function to skip to the previous closest situation time
def previous_time():
    global current_index
    if len(distance_index_list) > 0:
        current_index = (current_index - 1) % len(distance_index_list)
        specific_time_in_seconds = distance_index_list[current_index][0]  # Extract the correct time from closest situations
        seek_to_time(player, specific_time_in_seconds)
        time_label.config(text=f"Time: {specific_time_in_seconds} seconds")

# Create the Tkinter window
root = tk.Tk()
root.title("VLC Video Control")

# Pause Button
pause_button = Button(root, text="Pause", command=pause_video)
pause_button.pack(side=tk.LEFT)

# Play Button
play_button = Button(root, text="Play", command=play_video)
play_button.pack(side=tk.LEFT)

# Next Button
next_button = Button(root, text="Next", command=next_time)
next_button.pack(side=tk.LEFT)

# Previous Button
previous_button = Button(root, text="Previous", command=previous_time)
previous_button.pack(side=tk.LEFT)

# Close Button
close_button = Button(root, text="Close", command=close_video)
close_button.pack(side=tk.LEFT)

# Time Label
time_label = Label(root, text="Time: 0 seconds")
time_label.pack(side=tk.LEFT)

# Initialize index for closest situations
current_index = index_displayer_2.get_selected_index()


specific_time_in_seconds = one_match.iloc[current_index]['Time [s]']  # Access the first situation's time
print(f"Initial Time: {specific_time_in_seconds} seconds")
seek_to_time(player, specific_time_in_seconds)

# Start updating the time label
update_time_label()

# Keep the video running
root.mainloop() 